In [1]:
from pathlib import Path
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %matplotlib inline
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
import pandas as pd 
import geopandas as gpd
import pygeos
import seaborn as sns
import xarray as xr
import rioxarray as rio
from matplotlib.colors import LinearSegmentedColormap
import datetime
from datetime import timedelta
from scipy.spatial.distance import cdist
from math import dist
from scipy.stats import ttest_ind
from geopy import distance
from scipy import stats
from matplotlib.gridspec import GridSpec
from pylr2 import regress2
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
from matplotlib.ticker import MaxNLocator,FixedLocator
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde

# import shutil

/Users/weitao/anaconda3/lib/python3.10/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
lisVali = []
# trop and surface
for path in Path('./Hourly_MD/').rglob('*'):
# for path in Path('./Res_fus/').rglob('*'):
# for path in Path('./Res_fus5_DS/').rglob('*'):
    lisVali.append(path)
print(len(lisVali))
lisVali[:5]

8293


[PosixPath('Hourly_MD/0120230530'),
 PosixPath('Hourly_MD/0220230208'),
 PosixPath('Hourly_MD/0620210909'),
 PosixPath('Hourly_MD/0120210305'),
 PosixPath('Hourly_MD/0520230804')]

In [3]:
temp = pd.read_pickle(lisVali[0])

validata = pd.DataFrame(columns=list(temp.columns))
for i in lisVali:
    try:
        validata = pd.concat([validata,pd.read_pickle(i)])
    except:
        print(str(i))
validata

,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime,GEMS,AMFCloudFraction,SolarZenithAngle,ViewingZenithAngle,u,v,Wind_velocity,vcd0
56229,0.000219,37.4580,126.9510,1,2023053001,Seoul-SNU,2023-05-30 01:45:00,3.809300,0.297231,26.126650,43.360710,-4.368670,-0.473523,4.394258,6.242646
56228,0.000193,37.5644,126.9340,1,2023053001,Seoul,2023-05-30 01:45:00,13.680105,0.159725,26.224129,43.569859,-4.368670,-0.473523,4.394258,6.507455
56224,0.000024,43.5773,104.4191,1,2023053001,Dalanzadgad,2023-05-30 01:45:00,-6.947332,0.255835,42.462875,55.604958,6.455748,-2.654698,6.980266,3.110131
56227,0.000199,36.7769,126.4938,1,2023053001,Seosan,2023-05-30 01:45:00,11.115228,0.098210,26.099045,42.673416,-4.818448,0.067660,4.818923,7.002264
56223,0.000222,13.7847,100.5400,1,2023053001,Bangkok,2023-05-30 01:45:00,6.903467,0.214899,44.805023,35.767239,1.988636,0.544791,2.061909,4.745298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53053,0.000088,36.0661,140.1244,2,2023042502,Tsukuba,2023-04-25 02:45:00,4.031513,0.088655,23.094585,43.685707,-3.968970,3.858528,5.535428,5.301615
53047,0.000043,43.5773,104.4191,2,2023042502,Dalanzadgad,2023-04-25 02:45:00,-2.382923,0.126546,39.560558,55.634624,0.354300,5.643003,5.654114,2.706401
53046,0.000318,13.7847,100.5400,2,2023042502,Bangkok,2023-04-25 02:45:00,9.497755,0.125320,30.741302,35.780918,-0.566513,-0.252528,0.620248,5.569330
3660,0.000250,13.7847,100.5400,6,2021071806,Bangkok,2021-07-18 06:45:00,5.548569,0.243601,25.013895,35.795284,5.868202,2.149870,6.249618,5.507273


In [4]:
validata['ratio']=validata['vcd0']/validata['GEMS']
validata['VCD'] = validata['VCD(moles/m2)']* 6.022e23/1e4/1e15
validata['DVCD'] = validata['GEMS']-validata['vcd0']

In [5]:
# validata.to_pickle('./ValidataMD')

In [6]:
validata = pd.read_pickle('./ValidataMD')
validata

,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime,GEMS,AMFCloudFraction,SolarZenithAngle,ViewingZenithAngle,u,v,Wind_velocity,vcd0,ratio,VCD,DVCD,Month
56350,0.000219,37.4580,126.9510,1,2023053001,Seoul-SNU,2023-05-30 01:45:00,3.809300,0.297231,26.126650,43.360710,-4.368670,-0.473523,4.394258,6.242646,1.638791,13.213171,-2.433346,5
46552,0.000088,37.4580,126.9510,2,2023020802,Seoul-SNU,2023-02-08 02:45:00,3.825949,0.205094,54.045593,43.384853,3.227090,-3.708272,4.915830,3.645829,0.952922,5.270003,0.180120,2
6192,0.000357,37.4580,126.9510,6,2021090906,Seoul-SNU,2021-09-09 06:45:00,7.731202,0.090146,55.997890,43.432556,1.090323,1.383768,1.761709,8.357835,1.081052,21.510885,-0.626633,9
663,0.000085,37.4580,126.9510,1,2021030501,Seoul-SNU,2021-03-05 01:45:00,-1.100536,0.146820,50.591850,43.398949,1.798763,-3.302179,3.760311,4.158397,-3.778521,5.120085,-5.258932,3
39556,0.000054,37.4580,126.9510,1,2022102601,Seoul-SNU,2022-10-26 01:45:00,9.187380,0.203542,53.573238,43.459015,-0.623659,2.278356,2.362172,6.078684,0.661634,3.259227,3.108696,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14592,0.000041,36.0506,140.1202,2,2022021802,Tsukuba-NIES-West,2022-02-18 02:45:00,6.889956,0.164267,47.767956,43.653278,3.840096,-6.670386,7.697639,3.479825,0.505058,2.486996,3.410131,2
24667,0.000085,36.0506,140.1202,22,2022053122,Tsukuba-NIES-West,2022-05-31 22:45:00,4.097235,0.042868,49.611576,43.633640,2.472115,-1.921503,3.131059,5.562957,1.357735,5.124963,-1.465723,5
15415,0.000117,36.0506,140.1202,2,2022022702,Tsukuba-NIES-West,2022-02-27 02:45:00,6.977103,0.156220,44.469715,43.659405,9.363667,3.545740,10.019752,3.179444,0.455697,7.057483,3.797659,2
19402,0.000077,36.0506,140.1202,23,2022041023,Tsukuba-NIES-West,2022-04-10 23:45:00,5.483078,0.197787,48.364552,43.660511,1.089284,3.612895,3.773533,3.833920,0.699228,4.638506,1.649158,4


# Quality Control

In [9]:
validata2 = pd.DataFrame(columns=validata.columns)
for loca in validata['loca'].unique():
    try:
        temp = validata[validata['loca']==loca]
        Q1 = temp['GEMS'].quantile(0.25)
        Q3 = temp['GEMS'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        temp = temp[(temp['GEMS'] >= lower_bound) & (temp['GEMS'] <= upper_bound)]
        mean = temp['GEMS'].mean()
        std_dev = temp['GEMS'].std()
        print(loca + ' MEAN:'+ str(int(mean))+ ' S:'+str(int(std_dev)))
    #     print(std_dev)
        validata2 = pd.concat([validata2,temp])
    except:
        pass
validata2

Seoul-SNU MEAN:7 S:5
Seoul MEAN:6 S:4
Dalanzadgad MEAN:3 S:3
Bangkok MEAN:10 S:5
Seosan MEAN:6 S:4
Tsukuba-NIES MEAN:5 S:3
Incheon-ESC MEAN:7 S:4
Busan MEAN:7 S:4
Nagoya MEAN:5 S:3
Fukuoka MEAN:6 S:4
Tsukuba MEAN:5 S:3
Yokosuka MEAN:6 S:3
Tokyo-TMU MEAN:5 S:3
Kobe MEAN:6 S:3
Beijing-RADI MEAN:8 S:6
Singapore-NUS MEAN:10 S:4
Tsukuba-NIES-West MEAN:5 S:4


,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime,GEMS,AMFCloudFraction,SolarZenithAngle,ViewingZenithAngle,u,v,Wind_velocity,vcd0,ratio,VCD,DVCD,Month
56350,0.000219,37.4580,126.9510,1,2023053001,Seoul-SNU,2023-05-30 01:45:00,3.809300,0.297231,26.126650,43.360710,-4.368670,-0.473523,4.394258,6.242646,1.638791,13.213171,-2.433346,5
46552,0.000088,37.4580,126.9510,2,2023020802,Seoul-SNU,2023-02-08 02:45:00,3.825949,0.205094,54.045593,43.384853,3.227090,-3.708272,4.915830,3.645829,0.952922,5.270003,0.180120,2
6192,0.000357,37.4580,126.9510,6,2021090906,Seoul-SNU,2021-09-09 06:45:00,7.731202,0.090146,55.997890,43.432556,1.090323,1.383768,1.761709,8.357835,1.081052,21.510885,-0.626633,9
663,0.000085,37.4580,126.9510,1,2021030501,Seoul-SNU,2021-03-05 01:45:00,-1.100536,0.146820,50.591850,43.398949,1.798763,-3.302179,3.760311,4.158397,-3.778521,5.120085,-5.258932,3
39556,0.000054,37.4580,126.9510,1,2022102601,Seoul-SNU,2022-10-26 01:45:00,9.187380,0.203542,53.573238,43.459015,-0.623659,2.278356,2.362172,6.078684,0.661634,3.259227,3.108696,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23076,0.000120,36.0506,140.1202,1,2022051801,Tsukuba-NIES-West,2022-05-18 01:45:00,7.801951,0.059065,19.863686,43.631001,-0.521046,0.998883,1.126613,6.619256,0.848410,7.234831,1.182695,5
14592,0.000041,36.0506,140.1202,2,2022021802,Tsukuba-NIES-West,2022-02-18 02:45:00,6.889956,0.164267,47.767956,43.653278,3.840096,-6.670386,7.697639,3.479825,0.505058,2.486996,3.410131,2
24667,0.000085,36.0506,140.1202,22,2022053122,Tsukuba-NIES-West,2022-05-31 22:45:00,4.097235,0.042868,49.611576,43.633640,2.472115,-1.921503,3.131059,5.562957,1.357735,5.124963,-1.465723,5
15415,0.000117,36.0506,140.1202,2,2022022702,Tsukuba-NIES-West,2022-02-27 02:45:00,6.977103,0.156220,44.469715,43.659405,9.363667,3.545740,10.019752,3.179444,0.455697,7.057483,3.797659,2


In [10]:

validata2 = validata2[validata2['SolarZenithAngle']<60].copy()
validata2 = validata2[validata2['ViewingZenithAngle']<60].copy()

validata2['Month']=validata2['datetime'].dt.month
validata2
validata2

,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime,GEMS,AMFCloudFraction,SolarZenithAngle,ViewingZenithAngle,u,v,Wind_velocity,vcd0,ratio,VCD,DVCD,Month
56350,0.000219,37.4580,126.9510,1,2023053001,Seoul-SNU,2023-05-30 01:45:00,3.809300,0.297231,26.126650,43.360710,-4.368670,-0.473523,4.394258,6.242646,1.638791,13.213171,-2.433346,5
46552,0.000088,37.4580,126.9510,2,2023020802,Seoul-SNU,2023-02-08 02:45:00,3.825949,0.205094,54.045593,43.384853,3.227090,-3.708272,4.915830,3.645829,0.952922,5.270003,0.180120,2
6192,0.000357,37.4580,126.9510,6,2021090906,Seoul-SNU,2021-09-09 06:45:00,7.731202,0.090146,55.997890,43.432556,1.090323,1.383768,1.761709,8.357835,1.081052,21.510885,-0.626633,9
663,0.000085,37.4580,126.9510,1,2021030501,Seoul-SNU,2021-03-05 01:45:00,-1.100536,0.146820,50.591850,43.398949,1.798763,-3.302179,3.760311,4.158397,-3.778521,5.120085,-5.258932,3
39556,0.000054,37.4580,126.9510,1,2022102601,Seoul-SNU,2022-10-26 01:45:00,9.187380,0.203542,53.573238,43.459015,-0.623659,2.278356,2.362172,6.078684,0.661634,3.259227,3.108696,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23076,0.000120,36.0506,140.1202,1,2022051801,Tsukuba-NIES-West,2022-05-18 01:45:00,7.801951,0.059065,19.863686,43.631001,-0.521046,0.998883,1.126613,6.619256,0.848410,7.234831,1.182695,5
14592,0.000041,36.0506,140.1202,2,2022021802,Tsukuba-NIES-West,2022-02-18 02:45:00,6.889956,0.164267,47.767956,43.653278,3.840096,-6.670386,7.697639,3.479825,0.505058,2.486996,3.410131,2
24667,0.000085,36.0506,140.1202,22,2022053122,Tsukuba-NIES-West,2022-05-31 22:45:00,4.097235,0.042868,49.611576,43.633640,2.472115,-1.921503,3.131059,5.562957,1.357735,5.124963,-1.465723,5
15415,0.000117,36.0506,140.1202,2,2022022702,Tsukuba-NIES-West,2022-02-27 02:45:00,6.977103,0.156220,44.469715,43.659405,9.363667,3.545740,10.019752,3.179444,0.455697,7.057483,3.797659,2


In [ ]:
# validata2.to_pickle('ValidataMD2')